In [14]:
#Loading packages and functions for finding PRC2 binding in LMRs

import pandas as pd
import pyBigWig 

#please refer to s.table 1 to download the following bigwig files into the data/chip folder to run 

ez=pyBigWig.open('data/chip/ENCFF105JFX.bigWig') # EZH2 esc fold 
ez9=pyBigWig.open('data/chip/ENCFF000AVT.bigWig') # EZH2 esc fold  hg19
su=pyBigWig.open('data/chip/ENCFF224RZW.bigWig') # SUZ12 esc fold
su9=pyBigWig.open('data/chip/ENCFF027VOM.bigWig') # SUZ12 esc fold hg19

mez1=pyBigWig.open('data/chip/SRX2528911.bw') # EZH2 rep1 mesc chip-atlas
mez2=pyBigWig.open('data/chip/SRX2528912.bw') # EZH2 rep2 mesc chip-atlas
msu=pyBigWig.open('data/chip/SRX4338248.bw') # SUZ12 mesc chip-atlas

#remove ENCODE blacklist regions - bias high signal regions that are problematic in ChIP experiments
def bl(t,b):
    bl=pd.read_table('data/'+b,header=None)
    bl['r']=bl.apply(lambda x: set(range(x[1],x[2])),axis=1)
    bg=bl.groupby(0)['r'].apply(list)
    bg=bg.apply(lambda x:set.union(*x))   
    t['r']=t.apply(lambda x: set(range(x[1],x[2])),axis=1)
    t['bl']=t.apply(lambda x: len(x['r'].intersection(bg[x['ch']])),axis=1)
    t=t[t['bl']==0].drop('r',axis=1)
    return t

#Calculate PRC2 binding per LMR per sample - hg38
def prc(folder,cell,): 
    t=pd.read_csv('data/meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None,usecols=[0,1,2,4])
    t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
    t.columns=['ch','b','e','d']
    t['ez']=t.apply(lambda x:ez.stats(x[0],x[1],x[2])[0],axis=1)
    t['su']=t.apply(lambda x:su.stats(x[0],x[1],x[2])[0],axis=1)
    t['pr']=t[['ez','su']].mean(1)
    print(len(t))
    t=bl(t,'hg38-blacklist.v2.bed.gz') 
    return t

#Calculate PRC2 binding per LMR per sample - hg19
def prc9(folder,cell,): 
    t=pd.read_csv('data/meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None,usecols=[0,1,2,4])
    t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
    t.columns=['ch','b','e','d']
    t['ez']=t.apply(lambda x:ez9.stats(x[0],x[1],x[2])[0],axis=1)
    t['su']=t.apply(lambda x:su9.stats(x[0],x[1],x[2])[0],axis=1)
    t['pr']=t[['ez','su']].mean(1)
    print(len(t))
    t=bl(t,'hg19-blacklist.v2.bed.gz') 
    return t

#Calculate PRC2 binding per LMR per sample - mm10
def mprc(folder,cell): # PRC levels
    t=pd.read_csv('data/meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None,usecols=[0,1,2,4])
    t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
    t=t[~t[0].str.contains('random')].copy()
    t=t[~t[0].str.contains('Un')].copy()        
    t.columns=['ch','b','e','d']
    t=bl(t,'mm10-blacklist.v2.bed.gz')
    t['ez1']=t.apply(lambda x:mez1.stats(x[0],x[1],x[2])[0],axis=1)
    t['ez2']=t.apply(lambda x:mez2.stats(x[0],x[1],x[2])[0],axis=1)    
    t['su']=t.apply(lambda x:msu.stats(x[0],x[1],x[2])[0],axis=1)
    for c in ['ez1','ez2','su']:
        t[c]=t[c]/t[c].max()
    t['pr']=t[['ez1','ez2','su']].mean(1)   
    print(len(t))    
    return t

#Calculate DNAm percentage per LMR per sample 
def prcm(dft,folder,cells): # DNAm levels
    path='data/meth/'+folder+'/'
    for cell in cells:
        bs=[]
        print(cell,end=',')
        bw=pyBigWig.open(path+cell+'.bw')
        for i,r in dft.iterrows():
            if (i % 1000) == 0:print('.',end='')
            bs.append(bw.stats(r['ch'],r['b'],r['e'])[0])
        print()
        dft[cell]=bs
    dft=dft.dropna().copy().round(3)
    return dft

#Applies above functions
def pip(folder, cell,cells, hm='h',group='',hg=38):
    if hm=='h':
        if hg==38:
            t=prc(folder,cell)
        if hg==19:
            t=prc9(folder,cell)
    if hm=='m':
        t=mprc(folder,cell)    
    t=t.sort_values('pr').reset_index()
    t=prcm(t,folder,cells)
    return t

In [ ]:
# Figs 1,2,6 - Creating T-cell LMR table sorted by PRC2 binding (pr) (GSE79798 & GSE31263)
folder='t';cell='t';cells=['0','18','25','82','86','100']
t=pip(folder, cell,cells )

In [9]:
#saving T-cell LMR table 
t.to_csv('results/'+cell+'.csv')  

In [10]:
#calculating change in methylation between age 100 and age 0 and adding to results
t['delta']=t['100']-t['0']
t[['ch','b','e','d','pr','delta']].round(3).sort_values('pr',ascending=False).to_csv('results/'+cell+'_summary.csv',index=False)

In [12]:
# Figs 1&2 - Creating epidermis (skin) LMR table sorted by PRC2 binding (pr) (GSE52972)
folder='s';cell='s';cells=['18','23','25','74','75','83']
s=pip(folder, cell,cells)

76882
18,............................................................................
23,............................................................................
25,............................................................................
74,............................................................................
75,............................................................................
83,............................................................................


In [13]:
#saving epidermis LMR table 
s.to_csv('results/'+cell+'.csv')  

In [14]:
#calculating change in methylation between age 100 and age 0 and adding to results
s['delta']=s[['74','75','83']].mean(axis=1)-s[['18','23','25']].mean(axis=1)
s[['ch','b','e','d','pr','delta']].round(3).sort_values('pr',ascending=False).to_csv('results/'+cell+'_summary.csv',index=False)  

In [9]:
# Figs 2&4 - Creating mouse liver LMR table sorted by PRC2 binding (pr) (GSE89274)
folder='l';cell='l';cells=['SRR44738'+str(c) for c in range(55,55+8)]
l=pip(folder, cell,cells,hm='m')

53468
SRR4473855,......................................................
SRR4473856,......................................................
SRR4473857,......................................................
SRR4473858,......................................................
SRR4473859,......................................................
SRR4473860,......................................................
SRR4473861,......................................................
SRR4473862,......................................................


In [17]:
#calculating change in methylation between age 100 and age 0 and adding to results
l['delta']=l[['SRR44738'+str(c) for c in range(59,63)]].mean(1)-l[['SRR44738'+str(c) for c in range(55,59)]].mean(1)
l[['ch','b','e','d','pr','delta']].round(3).sort_values('pr',ascending=False).to_csv('results/'+cell+'_summary.csv',index=False)  

In [17]:
#saving mouse LMR table 
l.to_csv('results/'+cell+'.csv')  

In [5]:
# Fig 2 - Creating LMR table sorted by PRC2 binding (pr) for skin (sun damaged) samples (GSE52972)
cells=['SRR10429'+ i for i in  ['03','06','07','09','11','13']]+['SRR10429'+ i for i in  ['04','05','08','10','12','14']]
folder='s';cell='ss'
ss=pip(folder, cell,cells)

76882
SRR1042903,...........................................................................
SRR1042906,...........................................................................
SRR1042907,...........................................................................
SRR1042909,...........................................................................
SRR1042911,...........................................................................
SRR1042913,...........................................................................
SRR1042904,...........................................................................
SRR1042905,...........................................................................
SRR1042908,...........................................................................
SRR1042910,...........................................................................
SRR1042912,...........................................................................
SRR1042914,..........................

In [4]:
# Fig 5 - creating passaged fibroblast LMR table sorted by PRC2 binding (pr) (GSE79798)
folder='f';cell='fv';cells=['4','7','10','31','33']
f=pip(folder, cell,cells)

77733
4,...........................................................................
7,...........................................................................
10,...........................................................................
31,...........................................................................
33,...........................................................................


In [8]:
#calculating change in methylation between age 100 and age 0
f['delta']=f['33']-f['4']
f[['ch','b','e','d','pr','delta']].round(3).sort_values('pr',ascending=False).to_csv('results/'+cell+'_summary.csv',index=False)

In [5]:
#saving passaged fibroblast LMR table 
f.to_csv('results/'+cell+'.csv')  

In [13]:
#Ask Mahdi

folder='f/yo';cell='fyo';cells=['fet','y1','y2','y3','o1','o2','o3']
f=pip(folder, cell,cells)

73916
fet,.........................................................................
y1,.........................................................................
y2,.........................................................................
y3,.........................................................................
o1,.........................................................................
o2,.........................................................................
o3,.........................................................................


In [11]:
#Ask Mahdi

folder='f/sit';cell='fsit';cells=['fet','p0','p9','p18','90d']
f=pip(folder, cell,cells)

73916
fet,.........................................................................
p0,.........................................................................
p9,.........................................................................
p18,.........................................................................
90d,.........................................................................


In [6]:
#Calculating low methylated CpG PRC2 binding for mouse liver calorie restriction dataset (Fig 4a)
folder='l';cell='ly';cells=['SRR44738'+str(c) for c in range(55,55+16)]
ly=pip(folder, cell,cells,hm='m')

42660
SRR4473855,...........................................
SRR4473856,...........................................
SRR4473857,...........................................
SRR4473858,...........................................
SRR4473859,...........................................
SRR4473860,...........................................
SRR4473861,...........................................
SRR4473862,...........................................
SRR4473863,...........................................
SRR4473864,...........................................
SRR4473865,...........................................
SRR4473866,...........................................
SRR4473867,...........................................
SRR4473868,...........................................
SRR4473869,...........................................
SRR4473870,...........................................


In [7]:
# mouse dataset from here https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE102518 
# cant find it in the datatable, doublecheck if this one has been removed...
#Ask Mahdi

folder='e';cell='fr';cells=cells=['GSM279'+str(c) for c in range(4005,4015)]
fr=pip(folder, cell,cells,hm='m')

28725
GSM2794005,.............................
GSM2794006,.............................
GSM2794007,.............................
GSM2794008,.............................
GSM2794009,.............................
GSM2794010,.............................
GSM2794011,.............................
GSM2794012,.............................
GSM2794013,.............................
GSM2794014,.............................


In [ ]:
folder='e';cell='h';cells=['h']
h=pip(folder, cell,cells)

In [ ]:
# Fig 5 - The following LMR tables being created are cancer cell lines from TGCA

In [35]:
folder='c/c';cell='c';cells=['n','t1','t2']
c=pip(folder, cell,cells,hg=19)

58282
n,.........................................................
t1,.........................................................
t2,.........................................................


In [36]:
c.to_csv('results/c_'+cell+'.csv')  

In [37]:
folder='c/r';cell='r';cells=['n','t1','t2']
cr=pip(folder, cell,cells,hg=19)

67604
n,...................................................................
t1,...................................................................
t2,...................................................................


In [38]:
cr.to_csv('results/c_'+cell+'.csv')  

In [ ]:
folder='c/s';cell='s';cells=['n','t1','t2','t3','t4']
cs=pip(folder, cell,cells,hg=19)

In [41]:
cs.to_csv('results/c_'+cell+'.csv')  

In [5]:
folder='c/l';cell='lusc';cells=['n','t1','t2','t3','t4']
cl=pip(folder, cell,cells,hg=19)

59760
n,..........................................................
t1,..........................................................
t2,..........................................................
t3,..........................................................
t4,..........................................................


In [6]:
cl.to_csv('results/c_'+cell+'.csv')  

In [7]:
folder='c/luad';cell='luad';cells=['n','t1','t2','t3','t4','t5']
cld=pip(folder, cell,cells,hg=19)

62951
n,.............................................................
t1,.............................................................
t2,.............................................................
t3,.............................................................
t4,.............................................................
t5,.............................................................


In [8]:
cld.to_csv('results/c_'+cell+'.csv')  

In [9]:
folder='c/u';cell='u';cells=['n','t1','t2','t3','t4','t5']
cu=pip(folder, cell,cells,hg=19)

69865
n,....................................................................
t1,....................................................................
t2,....................................................................
t3,....................................................................
t4,....................................................................
t5,....................................................................


In [11]:
cu.to_csv('results/c_'+cell+'.csv')  

In [15]:
folder='c/blca';cell='blca';cells=['n','t1','t2','t3','t4','t5','t6']
cbl=pip(folder, cell,cells,hg=19)

64043
n,...............................................................
t1,...............................................................
t2,...............................................................
t3,...............................................................
t4,...............................................................
t5,...............................................................
t6,...............................................................


In [16]:
cbl.to_csv('results/c_'+cell+'.csv')  

In [17]:
folder='c/brca';cell='brca';cells=['n','t1','t2','t3','t4','t5']
cbr=pip(folder, cell,cells,hg=19)

62814
n,.............................................................
t1,.............................................................
t2,.............................................................
t3,.............................................................
t4,.............................................................
t5,.............................................................


In [18]:
cbr.to_csv('results/c_'+cell+'.csv')  

In [ ]:
folder='c/gbm';cell='gbm';cells=['n','t1','t2','t3','t4','t5']
s=pip(folder, cell,cells,ez=ez9,su=su9,group='c_')

In [ ]:
folder='l/sc';cell='lb';cells=['SRR3136670','SRR3136672','SRR3136631','SRR3136635','SRR3136654','SRR3136658']
s=pip(folder, cell,cells,hm='m',group='sc_')

In [4]:
# Fig 5b - Creating LMR table for brain oligodendrocytes (GSE107729)

folder='b';cell='b';cells=['GSM2877177','GSM2877184','GSM2877239','GSM2877242',
                           'GSM2877183','GSM2877238','GSM2877229']
s=pip(folder, cell,cells)

32286
GSM2877177,.............................
GSM2877184,.............................
GSM2877239,.............................
GSM2877242,.............................
GSM2877183,.............................
GSM2877238,.............................
GSM2877229,.............................


In [48]:
#Ask Mahdi
folder='e';cell='h';cells=['h']
h=pip(folder, cell,cells)

39517
h,.......................................


In [3]:
#Ask Mahdi - is this the same or different from the oligo LMRs above?

folder='b';cell='b';cells=['y1','y2','y3','o3','o4','o6','o7','o8','ol3','ol5','ol6','ol7','ol8']
re=pip(folder, cell,cells,hm='m')

13925
y1,..............
y2,..............
y3,..............
o3,..............
o4,..............
o6,..............
o7,..............
o8,..............
ol3,..............
ol5,..............
ol6,..............
ol7,..............
ol8,..............


In [36]:
#Ask Mahdi

folder='e';cell='em';
t=pd.read_csv('data/meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None)
t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
t=t[~t[0].str.contains('random')].copy()
t=t[~t[0].str.contains('Un')].copy()
t['d']=t[2]-t[1]
t.columns=['ch','b','e','d']
t=bl(t,'mm10-blacklist.v2.bed.gz')
t['ez1']=t.apply(lambda x:mez1.stats(x[0],x[1],x[2])[0],axis=1)
t['ez2']=t.apply(lambda x:mez2.stats(x[0],x[1],x[2])[0],axis=1)    
t['su']=t.apply(lambda x:msu.stats(x[0],x[1],x[2])[0],axis=1)
for c in ['ez1','ez2','su']:
    t[c]=t[c]/t[c].max()
t['pr']=t[['ez1','ez2','su']].mean(1)   
print(len(t))    

37919


In [ ]:
#Ask Mahdi

h=t.copy()

In [43]:
#Ask Mahdi

ht=h.sort_values('pr',ascending=False).head(1000)
ht[ht.d>100].to_csv('results/mm10_top_PRC2_d100.csv',index=False)

In [19]:
#For Fig6 PRC2-AgeIndex plots, running alternate functions only same tissue EZH2 ChIP (no SUZ12)

def prc_noSuz(folder,cell,): 
    t=pd.read_csv('data/meth/'+folder+'/'+cell+'.hmr',sep='\t',header=None,usecols=[0,1,2,4])
    t=t[~t[0].isin(['chrX','chrY','chrM'])].copy()
    t.columns=['ch','b','e','d']
    t['pr']=t.apply(lambda x:ez.stats(x[0],x[1],x[2])[0],axis=1)
    print(len(t))
    t=bl(t,'hg38-blacklist.v2.bed.gz') 
    return t

def pip2(folder, cell,cells, hm='h',group='',hg=38):
    if hm=='h':
        if hg==38:
            t=prc_noSuz(folder,cell)
        if hg==19:
            t=prc9(folder,cell)
    if hm=='m':
        t=mprc(folder,cell)    
    t=t.sort_values('pr').reset_index()
    t=prcm(t,folder,cells)
    return t

In [20]:
# Fig 6a - creating Neonatal fibroblast LMR table sorted by Neonatal fibroblast EZH2 binding (pr) (GSE253987)

ez=pyBigWig.open('data/chip/NEO_P2_Merged_fe.bw') # EZH2 NEO passage 2 fold-change binding HG38

folder='NeoFib';cell='NEO2_allPs';cells=["NEO2_P2_meth","NEO2_P5_meth","NEO2_P8_meth"]
neo=pip2(folder, cell,cells)

80176
NEO2_P2_meth,..............................................................................
NEO2_P5_meth,..............................................................................
NEO2_P8_meth,..............................................................................


In [21]:
#saving Neonatal fibroblast LMR table 
neo.to_csv('results/NEOP2s_methBWs_NEOMergeP2_FE.csv')

In [23]:
# Fig 6a - creating Old fibroblast LMR table sorted by Old fibroblast EZH2 binding (pr) (GSE253987)

ez=pyBigWig.open('data/chip/OLD_P2_Merged_fe.bw') # EZH2 OLD passage 2 fold-change binding HG38

folder='OldFib';cell='OLD3_allPs';cells=["OLD3_P2_meth","OLD3_P5_meth","OLD3_P8_meth"]
old=pip2(folder, cell,cells)

80573
OLD3_P2_meth,...............................................................................
OLD3_P5_meth,...............................................................................
OLD3_P8_meth,...............................................................................


In [24]:
#saving old fibroblast LMR table 

old.to_csv('results/OLDP2s_methBWs_OLDMergeP2_FE.csv')

In [25]:
#creating T-cell LMR table sorted by T-cell PRC2 binding (pr) (GSE79798, GSE31263 & GSE253773)

ez=pyBigWig.open('data/chip/EZH2_CD4_pooled_ENCinp_fe.bw') # EZH2 T-cell fold-change binding HG38

folder='t';cell='t';cells=['0','18','25','82','86','100']
t=pip2(folder, cell,cells)

57874
0,.........................................................
18,.........................................................
25,.........................................................
82,.........................................................
86,.........................................................
100,.........................................................


In [26]:
t.to_csv('results/t_cell_lmrs_CD4pooled_prc2_ENCinp_FE_meth.csv') 

In [27]:
#creating Neonatal fibroblast LMR table sorted by hESC PRC2 binding (pr) (GSE253985)

#loading hESC EZH2 and SUZ12 binding
ez=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_ezh2/ENCFF105JFX.bigWig') # EZH2 esc fold 
su=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_emb_suz/ENCFF224RZW.bigWig') # suz12 esc fold 

folder='NeoFib';cell='NEO2_allPs';cells=["NEO2_P2_meth","NEO2_P5_meth","NEO2_P8_meth"]
neo=pip(folder, cell,cells)

80176
NEO2_P2_meth,..............................................................................
NEO2_P5_meth,..............................................................................
NEO2_P8_meth,..............................................................................


In [28]:
neo.to_csv('results/NEOP2s_methBWs_hESCord_FE.csv')

In [29]:
#creating Old fibroblast LMR table sorted by hESC PRC2 binding (pr) (GSE253985)

#loading hESC EZH2 and SUZ12 binding
ez=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_ezh2/ENCFF105JFX.bigWig') # EZH2 esc fold 
su=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_emb_suz/ENCFF224RZW.bigWig') # suz12 esc fold 

folder='OldFib';cell='OLD3_allPs';cells=["OLD3_P2_meth","OLD3_P5_meth","OLD3_P8_meth"]
old=pip(folder, cell,cells)

80573
OLD3_P2_meth,...............................................................................
OLD3_P5_meth,...............................................................................
OLD3_P8_meth,...............................................................................


In [30]:
old.to_csv('results/OLDP2s_methBWs_hESCord_FE.csv')

In [33]:
#creating Neo and Old fibroblast LMR table sorted by hESC PRC2 binding (pr) (GSE253985)

#loading hESC EZH2 and SUZ12 binding
ez=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_ezh2/ENCFF105JFX.bigWig') # EZH2 esc fold 
su=pyBigWig.open('/Users/dsimps93/Project_FibroPass/OurLMRs_Mahdi_ver2/Recreate/data/chip_emb_suz/ENCFF224RZW.bigWig') # suz12 esc fold 

folder='Neo&OldFib';cell='ALL_fibropass';cells=["NEO2_P2_meth","NEO2_P5_meth","NEO2_P8_meth","OLD3_P2_meth","OLD3_P5_meth","OLD3_P8_meth"]
NewOld=pip(folder, cell,cells)

77273
NEO2_P2_meth,...........................................................................
NEO2_P5_meth,...........................................................................
NEO2_P8_meth,...........................................................................
OLD3_P2_meth,...........................................................................
OLD3_P5_meth,...........................................................................
OLD3_P8_meth,...........................................................................


In [34]:
NewOld.to_csv('results/Neo&Old_methBWs_hESCord_FE.csv')